<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras

In [3]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:190000]


In [9]:
pair in pairs

["I can't count the number of times Tom has complained about that.", "Je ne peux pas compter le nombre de fois où Tom s'est plaint à ce sujet.", 'CC-BY 2.0 (France) Attribution: tatoeba.org #1951444 (CK) & #3040332 (Julien_PDC)']


In [10]:
def clean_sentence(sentence):
    
    lower_case_sent = sentence.lower()
  
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [2]:
# from Book1 import words 
# print(words[0])
# for word in words:
#   z=word['English']+'\t'+word['French']
# #   raw_data3.append(z)
# text_file = open("sample.txt", "wt")
with open('fra_eng_sc_deeplearning.txt', 'w') as f:
  z="deep learning is a type of artificial intelligence derived from machine learning"+'\t'+"apprentissage profond est un type d'intelligence dérivé du apprentissage automatique"
  f.write("%s\n" % z)
  z="A recurrent neural network is a class of artificial neural networks "+'\t'+"Un réseau de neurones récurrent est une classe de réseaux de neurones artificiels"
  f.write("%s\n" % z)
  # for word in words:
  #   z=word['English'].split('\/')[0]+'\t'+word['French'].split('\/')[0]
  #   f.write("%s\n" % z)

f.close()


In [11]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [14]:
english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","") for pair in pairs]
french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").replace("\xad","").rstrip().lstrip() for pair in pairs]


fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))



french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 23
Maximum length english sentence: 16
french vocabulary is of 28313 unique words
English vocabulary is of 15381 unique words


Save french vocabulary



In [16]:
# Check language length

contant1=str(french_sentences)
file=open("french_sentences.txt","w+")
file.write(contant1)
file.close()


In [19]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,max_english_len, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)
print(max_french_len)
print(max_english_len)

23
16


In [18]:
print(max_english_len)

16


In [20]:
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

In [21]:
encoder = LSTM(128, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(128, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

In [ ]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=32, epochs=100)